In [ ]:
import numpy as np
import pandas as pd
import pickle
from scipy import linalg, optimize
from sklearn import linear_model, tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import timeit
from tqdm import tqdm
import xgboost as xgb

In [ ]:
# 1-d argmax
def fw1(x):
    d = len(x)
    m = np.max(x)
    for i in range(d):
        if x[i] == m:
            return i

# 2-d argmax
def fw2(x):
    d = x.shape
    m = np.max(x)
    for i in range(d[0]):
        for j in range(d[1]):
            if x[i, j] == m:
                return [i, j]

# Load Data

In [ ]:
df = pd.concat(pd.read_csv('Data/Russell.csv', chunksize = 10000))
ts = pd.read_csv('Data/tspredictors_1950.csv')
# Downcast to lower memory usage
df = df.apply(pd.to_numeric, downcast = 'float')
df = df.apply(pd.to_numeric, downcast = 'unsigned')
ts = ts.apply(pd.to_numeric, downcast = 'float')

df['year'] = df['date'].astype(str).str[0:4].astype('uint16')
df['month'] = df['date'].astype(str).str[4:6].astype('uint32')
df['year_month'] = df['date'].astype(str).str[0:6].astype('uint32')
years = np.arange(1957, 2017)

In [ ]:
macro = df.merge(ts, how = 'left', left_on = 'year_month', right_on = 'date', suffixes = ('_x', '')).iloc[:, 102:]
macro['C'] = 1
cols = ['C', 'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar'] #constant + 8 aggregate ts variable
macro = macro[cols]

y = df['ret']

# Random Forest Models

In [ ]:
def RF_MaxFeatures_Depth(x_train, y_train, x_val, y_val,mtrain, result_path, model_path):
    depth = range(1,6,1)  # depth
    max_features = range(20,65,5)
    ne = 100  
    r_oos = np.zeros((len(depth), len(max_features))) #r_oos is validation r2, same for all other models
    r_train = np.zeros((len(depth), len(max_features)))
    
    #Optimizing Model
    for n1, d in enumerate(tqdm(depth)): 
        for n2, mf in enumerate(max_features):
            model = RandomForestRegressor(max_depth = d, 
                                            n_estimators = ne, 
                                            max_features = mf,
                                            random_state = seed,
                                            n_jobs = -1)
            model.fit(x_train, y_train)
            
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
    
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_d = depth[int(fw2(r_oos)[0])]
    best_mf = max_features[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = RandomForestRegressor(max_depth = best_d, 
                                            n_estimators = ne,
                                            max_features = best_mf,
                                            random_state = seed,
                                            n_jobs = -1)
    optimized_model.fit(x_train, y_train)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_d, best_mf)

In [ ]:
def RF_MaxFeatures_MinSamples(x_train, y_train, x_val, y_val,mtrain, result_path, model_path):
    min_samples = range(1,11,1)  
    max_features = range(20,65,5)
    ne = 100  
    r_oos = np.zeros((len(min_samples), len(max_features)))
    r_train = np.zeros((len(min_samples), len(max_features)))
    
    #Optimizing Model
    for n1, ms in enumerate(tqdm(min_samples)): 
        for n2, mf in enumerate(max_features):
            model = RandomForestRegressor(max_depth = 2, 
                                            n_estimators = ne, 
                                            min_samples_leaf = ms,
                                            max_features = mf,
                                            random_state = seed,
                                            n_jobs = -1)
            model.fit(x_train, y_train)
            
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
    
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_ms = min_samples[int(fw2(r_oos)[0])]
    best_mf = max_features[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = RandomForestRegressor(max_depth = 2, 
                                            n_estimators = ne,
                                            min_samples_leaf = best_ms,
                                            max_features = best_mf,
                                            random_state = seed,
                                            n_jobs = -1)
    optimized_model.fit(x_train, y_train)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_ms, best_mf)

In [ ]:
def RF_MinSamples_Depth(x_train, y_train, x_val, y_val, mtrain, result_path, model_path):
    depth = range(1,6,1) 
    min_samples = range(1,11,1)
    ne = 100  
    r_oos = np.zeros((len(depth), len(min_samples)))
    r_train = np.zeros((len(depth), len(min_samples)))
    
    #Optimizing Model
    for n1, d in enumerate(tqdm(depth)): 
        for n2, mf in enumerate(min_samples):
            model = RandomForestRegressor(max_depth = d, 
                                            n_estimators = ne,
                                            min_samples_leaf = ms,
                                            max_features = 'sqrt',
                                            random_state = seed,
                                            n_jobs = -1)
            model.fit(x_train, y_train)
            
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
    
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_d = depth[int(fw2(r_oos)[0])]
    best_ms = min_samples[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = RandomForestRegressor(max_depth = best_d, 
                                            n_estimators = ne,
                                            min_samples_leaf = best_ms,
                                            max_features = 'sqrt',
                                            random_state = seed,
                                            n_jobs = -1)
    optimized_model.fit(x_train, y_train)
    print(optimized_model.feature_importances_)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_d, best_ms)

# Training, optimizing, and saving model

In [ ]:
seed = 7
for loop in range(34):
    train_idx = df.index[df['year'].isin(years[:(15 + loop)])]
    val_idx = df.index[df['year'].isin(years[(15 + loop):(25 + loop)])]

    #Using features without interaction with macro features   
    x_train = df.iloc[train_idx, 0:94]*macro.iloc[train_idx, 0][:, np.newaxis]
    x_val = df.iloc[val_idx, 0:94]*macro.iloc[val_idx, 0][:, np.newaxis]
    x_train = pd.concat((x_train, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[train_idx, :-1]), axis = 1)
    x_val = pd.concat((x_val, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[val_idx, :-1]), axis = 1)

    y_train = y[train_idx]
    y_val = y[val_idx]
    y_oos = y[oos_idx]
    mtrain = np.mean(y_train)
    
    print("X_train shape:",x_train.shape)
    print("X_val shape: ",x_val.shape)
    
    RF_MaxFeatures_Depth(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MF_D/','Models/RF/MF_D/')
    RF_MaxFeatures_MinSamples(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MF_MS/','Models/RF/MF_MS/')
    RF_MinSamples_Depth(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MS_D/','Models/RF/MS_D/')

    
    #Using Full Set Features
    x_train = df.iloc[train_idx, 0:94]*macro.iloc[train_idx, 0][:, np.newaxis]
    x_val = df.iloc[val_idx, 0:94]*macro.iloc[val_idx, 0][:, np.newaxis]
    for i in range(1, macro.shape[1]):
        x_train = pd.concat((x_train, df.iloc[train_idx, 0:94]*macro.iloc[train_idx, i][:, np.newaxis]), axis = 1)
        x_val = pd.concat((x_val, df.iloc[val_idx, 0:94]*macro.iloc[val_idx, i][:, np.newaxis]), axis = 1)
        
    x_train.columns = [a + '*' + b for b in macro.columns for a in df.columns[0:94]]
    x_val.columns = [a + '*' + b for b in macro.columns for a in df.columns[0:94]]
    
    x_train = pd.concat((x_train, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[train_idx, :-1]), axis = 1)
    x_val = pd.concat((x_val, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[val_idx, :-1]), axis = 1)
    
    print("X_train shape:",x_train.shape)
    print("X_val shape: ",x_val.shape)
    
    RF_MaxFeatures_Depth(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MF_D_Full/','Models/RF/MF_D_Full/')
    RF_MaxFeatures_MinSamples(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MF_MS_Full/','Models/RF/MF_MS_Full/')
    RF_MinSamples_Depth(x_train,y_train,x_val,y_val,mtrain,'Results/RF/MS_D_Full/','Models/RF/MS_D_Full/')

    
    
    


XGBoost Model Below

Parameters
n_estimators (int) – Number of gradient boosted trees. Equivalent to number of boosting rounds.

max_depth (int) – Maximum tree depth for base learners.

learning_rate (float) – Boosting learning rate (xgb’s “eta”)

verbosity (int) – The degree of verbosity. Valid values are 0 (silent) - 3 (debug).

objective (string or callable) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below).

booster (string) – Specify which booster to use: gbtree, gblinear or dart.

tree_method (string) – Specify which tree method to use. Default to auto. If this parameter is set to default, XGBoost will choose the most conservative option available. It’s recommended to study this option from parameters document.

n_jobs (int) – Number of parallel threads used to run xgboost.

gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.

min_child_weight (int) – Minimum sum of instance weight(hessian) needed in a child.

max_delta_step (int) – Maximum delta step we allow each tree’s weight estimation to be.

subsample (float) – Subsample ratio of the training instance.

colsample_bytree (float) – Subsample ratio of columns when constructing each tree.

colsample_bylevel (float) – Subsample ratio of columns for each level.

colsample_bynode (float) – Subsample ratio of columns for each split.

reg_alpha (float (xgb's alpha)) – L1 regularization term on weights

reg_lambda (float (xgb's lambda)) – L2 regularization term on weights

scale_pos_weight (float) – Balancing of positive and negative weights.

base_score – The initial prediction score of all instances, global bias.

random_state (int) –


eta typical value 0.01-0.2
min_child_weight controls overfitting, higher will be more generalization, minimum sum of weights
max_depth controls overfitting typical 3-10
gamma specifies minimum threshold for loss reduction
subsample, denote fraction of observations to be randomly samples, 0.5-1
lambda L2 regularization
alpha L1 Reg


In [ ]:
import xgboost as xgb

In [ ]:
def XGB_base_model(x_train, y_train, x_val, y_val, mtrain, result_path, model_path):
    learning_rates = [-1,-0.8,-0.6,-0.4,-0.2,0]
    max_depth = range(1,6,1)
    ne = 100  
    r_oos = np.zeros((len(learning_rates), len(max_depth)))
    r_train = np.zeros((len(learning_rates), len(max_depth)))
    
    for n1,llr in enumerate(tqdm(learning_rates)):
        for n2, d in enumerate(tqdm(max_depth)):
            lr = 10 ** llr
            model = xgb.XGBRegressor(max_depth = d, 
                                    n_estimators = ne, 
                                    learning_rate = lr,
                                    objective = 'reg:squarederror',
                                    reg_alpha = 0,
                                    reg_lambda = 0,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
            
            model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = False, early_stopping_rounds=25)
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
            
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_lr = learning_rates[int(fw2(r_oos)[0])]
    best_d = max_depth[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = xgb.XGBRegressor(max_depth = best_d, 
                                    n_estimators = ne, 
                                    learning_rate = 10**best_lr,
                                    objective = 'reg:squarederror',
                                    reg_alpha = 0,
                                    reg_lambda = 0,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
    
    optimized_model.fit(x_train, y_train,  eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = True, early_stopping_rounds=25)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_lr, best_d)

In [ ]:
#first run use max_depth = 3, lr = 0.2  range(0.0,0.5,0.05)
#second run use max_depth = 1, lr = 0.4 range(0.2,0.7,0.1)
#third run use max_depth = 3, lr = 0.2 range(0.2,0.7,0.1)
def XGB_alpha_lambda_model(x_train, y_train, x_val, y_val, mtrain, result_path, model_path):
    alphas = np.arange(0.2,0.7,0.1)
    lambdas = np.arange(0.2,0.7,0.1)
    ne = 100  
    r_oos = np.zeros((len(alphas), len(lambdas)))
    r_train = np.zeros((len(alphas), len(lambdas)))
    
    for n1,alpha in enumerate(tqdm(alphas)):
        for n2, l in enumerate(tqdm(lambdas)):
            model = xgb.XGBRegressor(max_depth = 3, 
                                    n_estimators = ne, 
                                    learning_rate = 0.2,
                                    objective = 'reg:squarederror',
                                    reg_alpha = alpha,
                                    reg_lambda = l,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
            
            model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = False, early_stopping_rounds=25)
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
            
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_alpha = alphas[int(fw2(r_oos)[0])]
    best_lambda = lambdas[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = xgb.XGBRegressor(max_depth = 3, 
                                    n_estimators = ne, 
                                    learning_rate = 0.2,
                                    objective = 'reg:squarederror',
                                    reg_alpha = best_alpha,
                                    reg_lambda = best_lambda,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
    
    optimized_model.fit(x_train, y_train,  eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = True, early_stopping_rounds=25)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_alpha, best_lambda)

In [ ]:
def XGB_sample_ratio_model(x_train, y_train, x_val, y_val, mtrain, result_path, model_path):
    sample_ratios = np.arange(0.5,1.0,0.1)
    
    #lambdas = np.arange(0.2,0.7,0.1)
    ne = 100  
    r_oos = np.zeros((len(sample_ratios), len(sample_ratios)))
    r_train =  np.zeros((len(sample_ratios), len(sample_ratios)))
    
    for n1, sample_ratio in enumerate(tqdm(sample_ratios)):
        for n2, col_ratio in enumerate(tqdm(sample_ratios)):
            model = xgb.XGBRegressor(max_depth = 1, 
                                    n_estimators = ne, 
                                    learning_rate = 0.4,
                                    objective = 'reg:squarederror',
                                    reg_alpha = 0.5,
                                    reg_lambda = 0.5,
                                    subsample = sample_ratio,
                                    colsample_bytree = col_ratio,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
            
            model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = False, early_stopping_rounds=20)
            y_hat = model.predict(x_val)
            r_oos[n1, n2] = 1 - sum(np.power(y_hat - y_val, 2))/sum(np.power(y_val - mtrain,2))
            r_train[n1, n2] = model.score(x_train, y_train)
            
    pd.DataFrame(r_oos).to_csv(result_path+'r2_oos'+str(years[25+loop])+'.csv')
    pd.DataFrame(r_train).to_csv(result_path+'r2_train'+str(years[25+loop])+'.csv')
    
    best_sample_ratio = sample_ratios[int(fw2(r_oos)[0])]
    best_col_ratio = sample_ratios[int(fw2(r_oos)[1])]
    
    #Build optimized model
    optimized_model = xgb.XGBRegressor(max_depth = 1, 
                                    n_estimators = ne, 
                                    learning_rate = 0.4,
                                    objective = 'reg:squarederror',
                                    reg_alpha = 0.5,
                                    reg_lambda = 0.5,
                                    subsample = best_sample_ratio,
                                    colsample_bytree = best_col_ratio,
                                    base_score = mtrain,
                                    random_state = 7,
                                    n_jobs = -1)
    
    optimized_model.fit(x_train, y_train,  eval_set=[(x_val, y_val)], eval_metric = 'rmse', verbose = True, early_stopping_rounds=20)
    pickle.dump(optimized_model, open(model_path+str(years[25+loop])+'.sav', 'wb'))
    
    return (best_sample_ratio, best_col_ratio)

In [ ]:
seed = 7
for loop in range(34):
    train_idx = df.index[df['year'].isin(years[:(15 + loop)])]
    val_idx = df.index[df['year'].isin(years[(15 + loop):(25 + loop)])]
    
    x_train = df.iloc[train_idx, 0:94]*macro.iloc[train_idx, 0][:, np.newaxis]
    x_val = df.iloc[val_idx, 0:94]*macro.iloc[val_idx, 0][:, np.newaxis]

    x_train = pd.concat((x_train, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[train_idx, :-1]), axis = 1)
    x_val = pd.concat((x_val, pd.get_dummies(df['sic2'].astype(str), prefix = 'sic2').iloc[val_idx, :-1]), axis = 1)

    y_train = y[train_idx]
    y_val = y[val_idx]
    mtrain = np.mean(y_train)
    
    print("X_train shape:",x_train.shape)
    print("X_val shape: ",x_val.shape)
    
    XGB_sample_ratio_model(x_train, y_train, x_val, y_val, mtrain,'Results/XGBoost/SR_CR/','Models/XGBoost/SR_CR/')